In [ ]:
!pip install langchain langchain-community langchain-google-genai requests duckduckgo-search python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63
  Attempting uninst

In [4]:
import os
import requests
from typing import Optional, Type
from langchain.tools import BaseTool, tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_google_genai import ChatGoogleGenerativeAI  # For Gemini
from dotenv import load_dotenv
import json
from google.colab import userdata

In [5]:
# Import userdata for Colab secrets
from google.colab import userdata

# Get API keys from Colab secrets
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')  # Your WeatherAPI key

# Verify Google API key is available
if GOOGLE_API_KEY:
    print("✅ Google API key found in Colab secrets!")
else:
    print("❌ Google API key not found. Please set GOOGLE_API_KEY in Colab secrets.")
    print("Go to Colab secrets tab (🔑) and add your Google API key.")

# Verify Weather API key
if WEATHER_API_KEY:
    print("✅ Weather API key configured!")
else:
    print("❌ Weather API key not found.")

print("✅ API keys configured successfully!")
print(f"Google API Key available: {'Yes' if GOOGLE_API_KEY else 'No'}")
print(f"Weather API Key available: {'Yes' if WEATHER_API_KEY else 'No'}")

✅ Google API key found in Colab secrets!
✅ Weather API key configured!
✅ API keys configured successfully!
Google API Key available: Yes
Weather API Key available: Yes


In [6]:
# Initialize Google Gemini 1.5 Flash (Free version)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Free model with generous limits
    temperature=0.1,           # Low temperature for consistent responses
    google_api_key=GOOGLE_API_KEY,
    convert_system_message_to_human=True  # Important for Gemini compatibility
)

print("✅ Google Gemini 1.5 Flash (Free) initialized successfully!")
print(f"Using model: Gemini 1.5 Flash")

# Test the LLM connection
try:
    test_response = llm.invoke("Hello, are you working?")
    print(f"✅ LLM test successful: {test_response.content[:50]}...")
except Exception as e:
    print(f"❌ LLM test failed: {str(e)}")
    print("💡 Tip: Make sure your Google API key is valid and has Gemini API access enabled")

✅ Google Gemini 1.5 Flash (Free) initialized successfully!
Using model: Gemini 1.5 Flash


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


✅ LLM test successful: Yes, I am working.  I'm ready to assist you with y...


In [7]:
@tool
def get_weather_forecast(city: str) -> str:
    """
    Fetch current weather and forecast for a given city using WeatherAPI.

    Args:
        city (str): The name of the city to get weather for

    Returns:
        str: Weather information including current conditions and forecast
    """
    try:
        # WeatherAPI endpoint for current weather and forecast
        url = f"http://api.weatherapi.com/v1/forecast.json"
        params = {
            'key': WEATHER_API_KEY,
            'q': city,
            'days': 3,  # 3-day forecast
            'aqi': 'no',  # Air quality index not needed
            'alerts': 'no'  # Weather alerts not needed
        }

        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        data = response.json()

        # Extract relevant weather information
        current = data['current']
        location = data['location']
        forecast = data['forecast']['forecastday']

        # Format the weather information
        weather_info = f"""
🌍 Weather for {location['name']}, {location['country']}:

📍 Current Conditions:
- Temperature: {current['temp_c']}°C ({current['temp_f']}°F)
- Condition: {current['condition']['text']}
- Feels like: {current['feelslike_c']}°C ({current['feelslike_f']}°F)
- Humidity: {current['humidity']}%
- Wind: {current['wind_kph']} km/h {current['wind_dir']}

📅 3-Day Forecast:
"""

        for day in forecast:
            date = day['date']
            day_info = day['day']
            weather_info += f"""
- {date}: {day_info['condition']['text']}
  High: {day_info['maxtemp_c']}°C, Low: {day_info['mintemp_c']}°C
  Rain chance: {day_info['daily_chance_of_rain']}%
"""

        return weather_info

    except requests.exceptions.RequestException as e:
        return f"❌ Error fetching weather data: {str(e)}"
    except KeyError as e:
        return f"❌ Error parsing weather data: Missing key {str(e)}"
    except Exception as e:
        return f"❌ Unexpected error: {str(e)}"

# Test the weather tool
print("✅ Weather tool created successfully!")
print("Testing weather tool...")
test_weather = get_weather_forecast("London")
print(test_weather[:200] + "..." if len(test_weather) > 200 else test_weather)

✅ Weather tool created successfully!
Testing weather tool...


<ipython-input-7-4199060511>:68: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  test_weather = get_weather_forecast("London")



🌍 Weather for London, United Kingdom:

📍 Current Conditions:
- Temperature: 18.1°C (64.6°F)
- Condition: Partly Cloudy
- Feels like: 18.1°C (64.6°F)
- Humidity: 83%
- Wind: 13.0 km/h SSW

📅 3-Day For...


In [8]:
# Initialize DuckDuckGo search tool (completely free, no API key required)
search_tool = DuckDuckGoSearchRun()

@tool
def get_tourist_attractions(city: str) -> str:
    """
    Search for top tourist attractions and interesting places to visit in a given city.

    Args:
        city (str): The name of the city to search attractions for

    Returns:
        str: Information about top tourist attractions in the city
    """
    try:
        # Create a focused search query for tourist attractions
        search_query = f"top tourist attractions {city} places to visit travel guide"

        # Perform the search
        search_results = search_tool.run(search_query)

        # Format the results for better readability
        formatted_results = f"""
🏛️ Top Tourist Attractions in {city}:

{search_results}

💡 Tips for visiting {city}:
- Check opening hours and ticket prices in advance
- Consider purchasing city passes for multiple attractions
- Use public transportation or walking when possible
- Try local cuisine and visit local markets
"""

        return formatted_results

    except Exception as e:
        return f"❌ Error searching for attractions: {str(e)}"

# Test the attractions tool
print("✅ Tourist attractions tool created successfully!")
print("Testing attractions tool...")
test_attractions = get_tourist_attractions("Paris")
print(test_attractions[:300] + "..." if len(test_attractions) > 300 else test_attractions)

✅ Tourist attractions tool created successfully!
Testing attractions tool...

🏛️ Top Tourist Attractions in Paris:

Guide to the best hotels and things to do in Paris. Maps, travel tips and more. These are the best things to do in Paris, according to our local experts. Read our complete Paris travel guide here , which includes: The 52 Best Hotels in Paris This list of the be...


In [9]:
# Create a list of tools available to the agent
tools = [get_weather_forecast, get_tourist_attractions]

# Create a prompt template for the travel assistant
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an intelligent Travel Assistant AI. Your role is to help users plan their trips by providing:

1. 🌤️ **Weather Information**: Current conditions and forecast for their destination
2. 🏛️ **Tourist Attractions**: Top places to visit and local recommendations

**Instructions:**
- Always be helpful, informative, and enthusiastic about travel
- Use the weather tool to get current weather and forecast
- Use the attractions tool to find interesting places to visit
- Provide practical travel advice when relevant
- Format your responses clearly with emojis and sections
- If a user asks about multiple cities, handle each one separately
- Always be encouraging and positive about travel experiences

**Available Tools:**
- get_weather_forecast: Get weather information for any city
- get_tourist_attractions: Find top tourist attractions in any city"""),

    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create the tool-calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Set to False to reduce output
    handle_parsing_errors=True,
    max_iterations=5  # Limit iterations to prevent infinite loops
)

print("✅ Travel Assistant Agent created successfully!")
print("🤖 Agent is ready to help with travel planning!")

✅ Travel Assistant Agent created successfully!
🤖 Agent is ready to help with travel planning!


In [10]:
def interactive_travel_assistant():
    """
    Interactive interface for the travel assistant
    """
    print("🌍 Welcome to your Intelligent Travel Assistant! 🌍")
    print("I can help you with weather information and tourist attractions for any destination.")
    print("Type 'quit' or 'exit' to end the conversation.\n")

    while True:
        try:
            # Get user input
            user_input = input("🏖️ Enter a destination you'd like to explore (or 'quit' to exit): ").strip()

            # Check if user wants to quit
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("✈️ Happy travels! Thanks for using the Travel Assistant!")
                break

            if not user_input:
                print("Please enter a valid destination name.")
                continue

            print(f"\n🔍 Gathering travel information for {user_input}...")
            print("=" * 60)

            # Get travel information
            result = travel_assistant(user_input)
            print(result)
            print("\n" + "=" * 60 + "\n")

        except KeyboardInterrupt:
            print("\n✈️ Happy travels! Thanks for using the Travel Assistant!")
            break
        except Exception as e:
            print(f"❌ An error occurred: {str(e)}")
            print("Please try again with a different destination.\n")

# Run the interactive assistant
interactive_travel_assistant()

🌍 Welcome to your Intelligent Travel Assistant! 🌍
I can help you with weather information and tourist attractions for any destination.
Type 'quit' or 'exit' to end the conversation.

🏖️ Enter a destination you'd like to explore (or 'quit' to exit): planning to visit munnar

🔍 Gathering travel information for planning to visit munnar...


> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `get_weather_forecast` with `{'city': 'Munnar'}`
responded: Planning a trip to Munnar is fantastic!  Let's get you all set with the information you need:

✈️ **Munnar Trip Planner** ✈️

First, let's check out the weather:

🌤️ **Weather Information:**




🌍 Weather for Munnar, India:

📍 Current Conditions:
- Temperature: 24.2°C (75.5°F)
- Condition: Light rain shower
- Feels like: 26.2°C (79.1°F)
- Humidity: 81%
- Wind: 11.9 km/h WSW

📅 3-Day Forecast:

- 2025-06-13: Moderate rain
  High: 24.8°C, Low: 18.7°C
  Rain chance: 96%

- 2025-06-14: Heavy rain
  High: 22.0°C, Low: 18.3°C
  Rain chance: 91%

- 2025-06-15: Moderate rain
  High: 20.5°C, Low: 18.0°C
  Rain chance: 83%


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `get_tourist_attractions` with `{'city': 'Munnar'}`



🏛️ Top Tourist Attractions in Munnar:

Here are the top 15 must-visit tourist attractions in Munnar, along with their locations and things to do. 1. Eravikulam National Park. Location: 13 km from Munnar Town Home to the endangered Nilgiri Tahr, Eravikulam National Park is a must-visit for wildlife enthusiasts. Here are the top 20 things to do in Munnar - your ultimate Munnar travel guide! Heads up: my trip to Kerala was a sponsored press trip with India Someday. But, all opinions and photos are my own. Click here for a review of their services, or see my full itinerary here! Discover top 10 best places to visit in Munnar, from tea gardens to waterfalls and national parks. ... 10 Best Places to Visit in Munnar: A Complete Guide to the Jewel of Kerala. ... Tips for Reaching Munnar. Best time to travel: The roads to Munnar are in good condition throughout the year, but the winter months (November to February) are ... In thi

...


Invoking: `get_weather_forecast` with `{'city': 'coimbatore'}`
responded: Planning a trip to Coimbatore is fantastic!  Let's get you all set with the information you need:

✈️ **Coimbatore Trip Planner** ✈️

First, let's check the weather:

🌤️ **Weather in Coimbatore:**




🌍 Weather for Coimbatore, India:

📍 Current Conditions:
- Temperature: 28.1°C (82.6°F)
- Condition: Partly cloudy
- Feels like: 30.5°C (86.9°F)
- Humidity: 74%
- Wind: 25.6 km/h WSW

📅 3-Day Forecast:

- 2025-06-13: Patchy rain nearby
  High: 28.2°C, Low: 23.4°C
  Rain chance: 83%

- 2025-06-14: Patchy rain nearby
  High: 25.9°C, Low: 22.9°C
  Rain chance: 89%

- 2025-06-15: Patchy rain nearby
  High: 26.3°C, Low: 22.6°C
  Rain chance: 87%


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `get_tourist_attractions` with `{'city': 'coimbatore'}`



🏛️ Top Tourist Attractions in coimbatore:

20. Race Course Walking Trail- Tourist Places in Coimbatore. A perfect place for fitness enthusiasts, this well-maintained walking trail is lined with trees and benches. Address: Race Course Road, Coimbatore; Timings: Open 24 hours; 21. Insect Museum, TNAU- Tourist Places in Coimbatore. A unique museum showcasing a vast collection of insect ... Here's your ultimate travel guide to the top 12 things to do in Coimbatore, each filled with ... and wildlife. The journey includes 40 hairpin bends offering spectacular views. You can visit attractions like Sholayar Dam and Balaji Temple or simply enjoy serene nature walks through lush plantations surrounded by mist and pristine mountain air ... For more adventure options, check out our guide on Discover the Best Places to Visit in Coimbatore! Cultural Treasures: Museums and Historical Landmarks. Coimbatore is rich in history and cul

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Hi there!  Get ready for an amazing trip to Coimbatore!  I've gathered some information to help you plan:

☀️ **Coimbatore Weather:**

Based on the latest forecast:

* **Current Conditions:**  It's currently 28.1°C (82.6°F) and partly cloudy in Coimbatore. It feels like 30.5°C (86.9°F) due to humidity.  Pack light clothing and stay hydrated!
* **3-Day Forecast:** There's a chance of patchy rain over the next few days. Pack an umbrella or raincoat just in case!


🏛️ **Coimbatore Tourist Attractions:**

Coimbatore offers a delightful mix of experiences! Here are some highlights:

* **Race Course Walking Trail:** Perfect for a morning or evening stroll, enjoying the greenery.
* **Insect Museum, TNAU:** A unique museum for those interested in the fascinating world of insects.
* **Marudamalai Hill Temple:** A significant pilgrimage site with stunning views. (Consider adding this to your itinerary if you're interested in religious sites).
* **Other Attractions:**  The information I have also